This will be one of my first projects, in this project, I aim to create a model that will detect whether mushrooms are poisonous or not.

I will use accuracy scoring,grid search, cross-validation and most of the classification models, and create an ensembled model.

Since I am a newbie in this field, I will have mistakes and shortcomings, so I ask you to indicate the places that you see incorrect or missing in the comments :)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load and Check Data

In [ ]:
df=pd.read_csv("../input/mushroom-classification/mushrooms.csv")

In [ ]:
df.head()

In [ ]:
df.dtypes

All variables are categorical variable.

In [ ]:
df["class"].replace(["p","e"],[1,0],inplace=True)

I marked poisonous ones as 1 in order to make achieve my goal by using recall scoring.

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

There is no missing value so it's good for us. We don't have to spend effort on this and since all of them categorical we don't have any outliers.

# Categorical Data Analysis with Visualization

In [ ]:
sns.countplot("class",data=df)

The number of both classes is about the same, but edible ones are slightly more.

In [ ]:
for i in [ 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat']:
    
    plt.subplots()
    sns.countplot(x=i,hue="class",data=df,order = df[i].value_counts().index,palette=['green',"red"])
    plt.legend()
    plt.show()

As we can see some variables are similar and some of are not correlated much with target like gill-attachment and veil-type. these two variables does not give information about target so we can drop them.

# Check and Drop Values

In [ ]:
plt.figure()
sns.countplot(x="class",data=df)
plt.figure()
sns.countplot(x="veil-type",hue="class",data=df,order = df["veil-type"].value_counts().index,palette=['green',"red"])
plt.legend()
plt.show()

As we can see veil-type's behavior does not give information about class so we can drop it.

In [ ]:
df.drop(["veil-type"],axis=1,inplace=True)

In [ ]:
plt.figure()
sns.countplot(x="class",data=df)
plt.figure()
sns.countplot(x="gill-attachment",hue="class",data=df,order = df["gill-attachment"].value_counts().index,palette=['green',"red"])
plt.legend()
plt.show()

And gill-attachment's behavior is similar to veil-type's one and it does not give information so we can drop it too.

In [ ]:
df.drop(["gill-attachment"],axis=1,inplace=True)

In [ ]:
plt.figure()
sns.countplot(x="odor",hue="class",data=df,order = df["odor"].value_counts().index,palette=['green',"red"])
plt.legend()
plt.show()

As we can see odor variable is highly correlated with class so it can cause data leakage. Maybe odor name named after class name so it will be better if we drop that variable in order to prevent data leakage.

In [ ]:
df.drop(["odor"],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
x=pd.get_dummies(df,columns=['cap-shape', 'cap-surface', 'cap-color', 'bruises',
       'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root',
       'stalk-surface-above-ring', 'stalk-surface-below-ring',
       'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color',
       'ring-number', 'ring-type', 'spore-print-color', 'population',
       'habitat']).drop("class",axis=1)
y=df["class"]

Since all of my variables are categorical I changed every x variable to make suitable for my model.

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.25,random_state=42)

# Modelling

-I will use DecisionTreeClassifier,SVC,RandomForestClassifier, LogisticRegression,KNeighborsClassifier

-I want to create a model that detects all poisonous mushrooms since no one would eat them.  I will use "Recall Scoring" because  I labelled poisonous mushrooms "1" and I want my model to detect all of the positive labels. That's why I will use "Recall Scoring".

-I will use GridSearch to find the best parameters for all of the classifiers.

-I will score them by using cross-validation.

-I will find the best 3 or 4 models and I will create an ensembled model then I will finish.

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler,QuantileTransformer
from sklearn.model_selection import GridSearchCV, cross_val_score,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score,f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
Confusion_Matrices=[]
Classifiers=[]
Scores=[]
Best_Parameters=[]
Cross_Val_Test_Scores=[]
random_state=42

In [ ]:
dt_param_grid = {"min_samples_split" : range(20,400,20),
                "max_depth": range(2,18,2)}


clf=GridSearchCV(DecisionTreeClassifier(random_state = random_state),param_grid=dt_param_grid,cv=StratifiedKFold(n_splits = 5),scoring="accuracy")
clf.fit(x_train,y_train)
Scores.append(clf.best_score_)
Confusion_Matrices.append(confusion_matrix(y_test, clf.predict(x_test)))
Classifiers.append("Decision Tree")
Cross_Val_Test_Scores.append(cross_val_score(clf, x_test, y_test,scoring="accuracy", cv=5).mean())
Best_Parameters.append(clf.best_estimator_)

In [ ]:
svc_param_grid = {"kernel" : ["rbf"],
                 "gamma": [0.001, 0.01, 0.1, 1],
                 "C": [1,10,50,100,200,1000]}



clf=GridSearchCV(SVC(random_state = random_state),param_grid=svc_param_grid,cv=StratifiedKFold(n_splits = 5),scoring="accuracy")
clf.fit(x_train,y_train)
Scores.append(clf.best_score_)
Confusion_Matrices.append(confusion_matrix(y_test, clf.predict(x_test)))
Classifiers.append("SVC")
Cross_Val_Test_Scores.append(cross_val_score(clf, x_test, y_test,scoring="accuracy", cv=5).mean())
Best_Parameters.append(clf.best_estimator_)

In [ ]:
rf_param_grid = {"max_features": [1,3,10],
                "min_samples_split":[2,3,10],
                "min_samples_leaf":[1,3,10],
                "bootstrap":[False],
                "n_estimators":[100,300],
                "criterion":["gini"]}



clf=GridSearchCV(RandomForestClassifier(random_state = random_state),param_grid=rf_param_grid,cv=StratifiedKFold(n_splits = 5),scoring="accuracy")
clf.fit(x_train,y_train)
Scores.append(clf.best_score_)
Confusion_Matrices.append(confusion_matrix(y_test, clf.predict(x_test)))
Classifiers.append("Random Forest")
Cross_Val_Test_Scores.append(cross_val_score(clf, x_test, y_test,scoring="accuracy", cv=5).mean())
Best_Parameters.append(clf.best_estimator_)

In [ ]:
logreg_param_grid = {"C":np.logspace(-3,3,7)}
clf=GridSearchCV(LogisticRegression(random_state = random_state),param_grid=logreg_param_grid,cv=StratifiedKFold(n_splits = 5),scoring="accuracy")
clf.fit(x_train,y_train)
Scores.append(clf.best_score_)
Confusion_Matrices.append(confusion_matrix(y_test, clf.predict(x_test)))
Classifiers.append("Logistic Regression")
Cross_Val_Test_Scores.append(cross_val_score(clf, x_test, y_test,scoring="accuracy", cv=5).mean())
Best_Parameters.append(clf.best_estimator_)


In [ ]:
knn_param_grid = {"n_neighbors": np.linspace(1,19,10, dtype = int).tolist(),
                 "weights": ["uniform","distance"],
                 "metric":["euclidean","manhattan"]}

clf=GridSearchCV(KNeighborsClassifier(),param_grid=knn_param_grid,cv=StratifiedKFold(n_splits = 5),scoring="accuracy")
clf.fit(x_train,y_train)
Scores.append(clf.best_score_)
Confusion_Matrices.append(confusion_matrix(y_test, clf.predict(x_test)))
Classifiers.append("KNN")
Cross_Val_Test_Scores.append(cross_val_score(clf, x_test, y_test,scoring="accuracy", cv=5).mean())
Best_Parameters.append(clf.best_estimator_)

In [ ]:
graph_data= pd.DataFrame(list(zip(Classifiers,Cross_Val_Test_Scores)),columns =['Classifiers', 'Cross_Val_Test_Scores']) 
graph_data=graph_data.sort_values("Cross_Val_Test_Scores",ascending=False)
standartscaler=QuantileTransformer()
graph_data["Scaled Scores"]=standartscaler.fit_transform(graph_data["Cross_Val_Test_Scores"].values.reshape(-1,1))
plt.figure(figsize=(16,8))
sns.barplot(x=graph_data["Classifiers"],y=graph_data["Scaled Scores"])
plt.title("Scores with scaled data")
plt.figure(figsize=(16,8))
sns.barplot(x=graph_data["Classifiers"],y=graph_data['Cross_Val_Test_Scores'])
plt.title("Scores with real data")

In [ ]:
graph_data.head()

All of models that I used with their best parameters gave more than 99% accuracy. But still I will use an ensemble model for both creating a better model and avoid overfitting as much as possible.

In [ ]:
for i in range(len(Cross_Val_Test_Scores)):
    plt.subplots()
    sns.heatmap(Confusion_Matrices[i],annot=True,fmt='d')
    plt.title("Confusion Matrix of {}".format(Classifiers[i]))

Still I cannot see any problem from any of the models.

# Ensemble Modelling

In [ ]:
votingC = VotingClassifier(estimators = [("knn",Best_Parameters[4]),
                                        ("lr",Best_Parameters[3]),
                                        ("rf",Best_Parameters[2])],
                                        voting = "soft", n_jobs = -1)
votingC = votingC.fit(x_train, y_train)
print(accuracy_score(votingC.predict(x_test),y_test))

At the end we created a model that has 100% accuracy.

I hope you liked it, please do not forget to mention the wrong and missing places in the comments.